<a href="https://colab.research.google.com/github/Gitanjali988/Data-Growth-Analysis/blob/main/In_Tandem_Data_Growth_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pandasql -q

  Preparing metadata (setup.py) ... done


In [ ]:
import pandas as pd
from pandasql import sqldf



In [ ]:
import pandas as pd
from pandasql import sqldf

# Load the CSV file into a pandas DataFrame
from google.colab import files
uploaded = files.upload()
csv_file_path = 'orders.csv'
orders_df = pd.read_csv(csv_file_path)

# Make the DataFrame available for querying with pandasql
# The variable name of the DataFrame becomes the table name in SQL
globals().update(locals())

# Example SQL query
query = """
SELECT
    *
FROM
    orders_df
LIMIT 10;
"""

# Execute the SQL query
result_df = sqldf(query)

# Display the results
display(result_df)

Saving orders.csv to orders.csv


,invoice_id,line_item_id,user_id,item_id,item_name,item_category,price,created_at,paid_at
0,192320.0,83118.0,178481.0,3526.0,digital apparatus,apparatus,330.0,2017-06-28 21:14:25,2017-06-27 21:19:39
1,192320.0,207309.0,178481.0,1514.0,miniature apparatus cleaner,apparatus,99.0,2017-06-28 21:14:25,2017-06-27 21:19:39
2,192320.0,392027.0,178481.0,3712.0,miniature apparatus cleaner,apparatus,99.0,2017-06-28 21:14:25,2017-06-27 21:19:39
3,80902.0,243831.0,154133.0,3586.0,reflective instrument,instrument,57.2,2016-10-09 06:57:30,2016-10-07 10:08:10
4,80902.0,399806.0,154133.0,1061.0,extra-strength instrument charger,instrument,17.6,2016-10-09 06:57:30,2016-10-07 10:08:10
5,211144.0,19924.0,119514.0,550.0,miniature apparatus refill,apparatus,132.0,2016-07-25 08:01:15,2016-07-23 11:32:43
6,123735.0,154006.0,164581.0,982.0,miniature module carrying_case,module,33.0,2016-11-24 07:35:34,2016-11-26 20:52:06
7,123735.0,66700.0,164581.0,858.0,industrial-strength module,module,300.0,2016-11-24 07:35:34,2016-11-26 20:52:06
8,123735.0,321246.0,164581.0,3363.0,rechargable module cleaner,module,78.0,2016-11-24 07:35:34,2016-11-26 20:52:06
9,249245.0,292068.0,232806.0,2344.0,prize-winning tool,tool,27.5,2018-02-07 05:23:44,2018-02-08 19:07:55


# **Question 1: Cohort Retention Analysis**

In [ ]:
# Load the CSV
orders_df = pd.read_csv('/content/orders.csv', parse_dates=["created_at", "paid_at"])

# Format for SQL
orders_df["created_month"] = orders_df["created_at"].dt.strftime("%Y-%m")
orders_df["created_date"] = orders_df["created_at"].dt.date

Data Overview:

In [ ]:
orders_df.info()
orders_df.describe()
orders_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47402 entries, 0 to 47401
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   invoice_id        47402 non-null  float64       
 1   line_item_id      47402 non-null  float64       
 2   user_id           47402 non-null  float64       
 3   item_id           47402 non-null  float64       
 4   item_name         47402 non-null  object        
 5   item_category     47402 non-null  object        
 6   price             47402 non-null  float64       
 7   created_at        47402 non-null  datetime64[ns]
 8   paid_at           47402 non-null  datetime64[ns]
 9   created_month     47402 non-null  object        
 10  created_date      47402 non-null  object        
 11  cohort_month      47402 non-null  object        
 12  order_month       47402 non-null  object        
 13  order_year        47402 non-null  int32         
 14  order_month_num   4740

,invoice_id,line_item_id,user_id,item_id,item_name,item_category,price,created_at,paid_at,created_month,created_date,cohort_month,order_month,order_year,order_month_num,cohort_year,cohort_month_num,cohort_index
0,192320.0,83118.0,178481.0,3526.0,digital apparatus,apparatus,330.0,2017-06-28 21:14:25,2017-06-27 21:19:39,2017-06,2017-06-28,2017-06,2017-06,2017,6,2017,6,1
1,192320.0,207309.0,178481.0,1514.0,miniature apparatus cleaner,apparatus,99.0,2017-06-28 21:14:25,2017-06-27 21:19:39,2017-06,2017-06-28,2017-06,2017-06,2017,6,2017,6,1
2,192320.0,392027.0,178481.0,3712.0,miniature apparatus cleaner,apparatus,99.0,2017-06-28 21:14:25,2017-06-27 21:19:39,2017-06,2017-06-28,2017-06,2017-06,2017,6,2017,6,1
3,80902.0,243831.0,154133.0,3586.0,reflective instrument,instrument,57.2,2016-10-09 06:57:30,2016-10-07 10:08:10,2016-10,2016-10-09,2016-10,2016-10,2016,10,2016,10,1
4,80902.0,399806.0,154133.0,1061.0,extra-strength instrument charger,instrument,17.6,2016-10-09 06:57:30,2016-10-07 10:08:10,2016-10,2016-10-09,2016-10,2016-10,2016,10,2016,10,1


***Part 1: Standard cohort table: Cohort month, cohort size, and retention rates for months 1-12***

*Step 1. Find the cohort month for each user-their first order month*

In [ ]:
cohort_sql = """
SELECT
    user_id,
    MIN(strftime('%Y-%m', created_at)) AS cohort_month
FROM orders_df
GROUP BY user_id
"""
user_cohorts = sqldf(cohort_sql)

# Merge cohort_month into orders_df for SQL use
orders_df = orders_df.merge(user_cohorts, on="user_id")

*Step 2. Add order_month (created_at month)*

In [ ]:
orders_df["order_month"] = orders_df["created_at"].dt.strftime("%Y-%m")


*Step 3. Calculate cohort_index = month difference between cohort and current order.*

Adding numeric year and month for calculation

In [ ]:
orders_df["order_year"] = orders_df["created_at"].dt.year
orders_df["order_month_num"] = orders_df["created_at"].dt.month
orders_df["cohort_year"] = pd.to_datetime(orders_df["cohort_month"]).dt.year
orders_df["cohort_month_num"] = pd.to_datetime(orders_df["cohort_month"]).dt.month

orders_df["cohort_index"] = (
    (orders_df["order_year"] - orders_df["cohort_year"]) * 12 +
    (orders_df["order_month_num"] - orders_df["cohort_month_num"]) + 1
)

*Step 4. Create cohort table (user counts per cohort_month and cohort_index)*


In [ ]:
retention_sql = """
SELECT
    cohort_month,
    cohort_index,
    COUNT(DISTINCT user_id) AS retained_users
FROM orders_df
GROUP BY cohort_month, cohort_index
ORDER BY cohort_month, cohort_index
"""
retention_counts = sqldf(retention_sql)

*Step 5. Calculate cohort size (users in cohort_index = 1)*


In [ ]:
cohort_size_sql = """
SELECT
    cohort_month,
    retained_users AS cohort_size
FROM retention_counts
WHERE cohort_index = 1
"""
cohort_sizes = sqldf(cohort_size_sql)

In [ ]:
#Merge retention with cohort size
retention_merged = pd.merge(retention_counts, cohort_sizes, on="cohort_month")

# Calculate retention rate
retention_merged["retention_rate"] = (retention_merged["retained_users"] / retention_merged["cohort_size"]).round(3)

# Pivot for cohort table format
cohort_pivot = retention_merged.pivot(index="cohort_month", columns="cohort_index", values="retention_rate").fillna(0)
cohort_pivot.style.format("{:.0%}").set_caption("Standard Cohort Retention Table")

cohort_index,1,2,3,4,5,6,7,8
cohort_month,,,,,,,,
2013-03,100%,0%,0%,0%,0%,0%,0%,0%
2013-04,100%,0%,0%,0%,12%,12%,0%,0%
2013-05,100%,0%,0%,0%,5%,0%,0%,0%
2013-06,100%,0%,0%,0%,0%,4%,0%,0%
2013-07,100%,2%,2%,6%,0%,0%,0%,0%
2013-08,100%,0%,2%,0%,4%,0%,0%,0%
2013-09,100%,0%,4%,1%,1%,0%,1%,0%
2013-10,100%,0%,4%,4%,1%,1%,2%,0%
2013-11,100%,5%,1%,3%,1%,1%,0%,0%


*   Cohort Month is the user's first purchase month — all subsequent behavior is tracked relative to this.

*   Cohort Index represents how many months after the initial purchase the customer returned.

*   This view helps understand customer retention decay and the "stickiness" of users acquired each month.

**Insights: Cohort Retention Trends**

*   Most cohorts experience a sharp drop in retention after Month 1, indicating that many users do not return after their first purchase.

*   Retention stabilizes around Months 3-5, with a small group of loyal users consistently returning.

*   Later cohorts (e.g recent months) show slightly higher early retention, possibly due to improved acquisition or onboarding strategies.

***Part 2: Resurrection analysis: Identify customers who return after being inactive for 2+ months and calculate "resurrection rates" by cohort***

A customer is “resurrected” if they:

1.   Made a purchase in month M
2.   Had no purchase in the previous 2+ months
3.   We calculate the resurrection rate = resurrected users / lost users from that cohort

*Step 1: Prepare simplified table of user monthly activity*


In [ ]:
orders_df["order_month"] = pd.to_datetime(orders_df["created_at"]).dt.to_period("M").astype(str)

# Create a monthly activity table: one row per user per active month
activity_sql = """
SELECT DISTINCT
    user_id,
    cohort_month,
    order_month
FROM orders_df
ORDER BY user_id, order_month
"""
user_monthly = sqldf(activity_sql)


*Step 2: Sort by user & month to calculate gaps between orders*


In [ ]:
user_monthly["order_month_dt"] = pd.to_datetime(user_monthly["order_month"])
user_monthly = user_monthly.sort_values(["user_id", "order_month_dt"])



In [ ]:
# Step 3: Lag the previous order month to find gaps
user_monthly["prev_order_month_dt"] = user_monthly.groupby("user_id")["order_month_dt"].shift(1)



In [ ]:
# Step 4: Calculate inactivity gap in months
user_monthly["months_since_last_order"] = ((user_monthly["order_month_dt"].dt.year - user_monthly["prev_order_month_dt"].dt.year) * 12 +
                                           (user_monthly["order_month_dt"].dt.month - user_monthly["prev_order_month_dt"].dt.month))


In [ ]:
# Step 5: Flag resurrected users (gap >= 3 months)
user_monthly["resurrected"] = user_monthly["months_since_last_order"] >= 3


*Step 6: Get resurrection count by cohort and month*

In [ ]:
resurrected_df = user_monthly[user_monthly["resurrected"] == True]
resurrection_summary = resurrected_df.groupby(["cohort_month", "order_month"]).agg(
    resurrected_users=('user_id', 'nunique')
).reset_index()

# Display resurrection table
resurrection_summary.head()

,cohort_month,order_month,resurrected_users
0,2013-04,2013-08,1
1,2013-04,2013-09,1
2,2013-05,2013-09,1
3,2013-06,2013-11,1
4,2013-07,2013-10,3


from matplotlib import pyplot as plt
_df_0['resurrected_users'].plot(kind='hist', bins=20, title='resurrected_users')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
_df_1.groupby('cohort_month').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
_df_2.groupby('order_month').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  counted = (series['resurrected_users']
                .value_counts()
              .reset_index(name='counts')
              .rename({'index': 'resurrected_users'}, axis=1)
              .sort_values('resurrected_users', ascending=True))
  xs = counted['resurrected_users']
  ys = counted['counts']
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = _df_3.sort_values('resurrected_users', ascending=True)
for i, (series_name, series) in enumerate(df_sorted.groupby('cohort_month')):
  _plot_series(series, series_name, i)
  fig.legend(title='cohort_month', bbox_to_anchor=(1, 1), loc='upper left')
sns.despine(fig=fig, ax=ax)
plt.xlabel('resurrected_users')
_ = plt.ylabel('count()')

...

from matplotlib import pyplot as plt
import seaborn as sns
import pandas as pd
plt.subplots(figsize=(8, 8))
df_2dhist = pd.DataFrame({
    x_label: grp['order_month'].value_counts()
    for x_label, grp in _df_6.groupby('cohort_month')
})
sns.heatmap(df_2dhist, cmap='viridis')
plt.xlabel('cohort_month')
_ = plt.ylabel('order_month')

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_7['cohort_month'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_7, x='resurrected_users', y='cohort_month', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_8['order_month'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_8, x='resurrected_users', y='order_month', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

This highlights how win-back strategies perform.

Resurrection rate can be used to determine:

*   Which months had successful re-engagement
*   What % of “**churned**” users return
*   The "**tail loyalty**" of  cohorts

***Resurrection Rate by Cohort (Percentage)***

Calculate the resurrection rate = resurrected users / previously churned users from the same cohort.

A churned user is someone who stopped purchasing for at least 2 months. We approximate "churn base" as users who made a purchase but were inactive before resurrection.



In [ ]:
# 1. Get last active month for each user before resurrection
resurrected_df["prev_order_month"] = resurrected_df["prev_order_month_dt"].dt.to_period("M").astype(str)

# 2. Count total resurrected users per cohort_month and resurrection_month
resurrection_counts = resurrected_df.groupby(["cohort_month", "order_month"]).agg(
    resurrected_users=('user_id', 'nunique')
).reset_index()

# 3. Estimate churn base: total unique users in that cohort before resurrection month
# Filter to include only order_months before resurrection date
churn_base_df = user_monthly.copy()
churn_base_df["order_month"] = pd.to_datetime(churn_base_df["order_month"])
resurrection_counts["order_month_dt"] = pd.to_datetime(resurrection_counts["order_month"])

# Join to get pre-resurrection users
churn_base = []
for i, row in resurrection_counts.iterrows():
    cohort = row["cohort_month"]
    cutoff = row["order_month_dt"]
    eligible_users = churn_base_df[
        (churn_base_df["cohort_month"] == cohort) &
        (churn_base_df["order_month"] < cutoff)
    ]["user_id"].nunique()

    churn_base.append(eligible_users)

resurrection_counts["churn_base"] = churn_base

# 4. Calculate resurrection rate
resurrection_counts["resurrection_rate"] = (resurrection_counts["resurrected_users"] / resurrection_counts["churn_base"]).round(3)

# 5. Clean final output
resurrection_output = resurrection_counts[["cohort_month", "order_month", "resurrected_users", "churn_base", "resurrection_rate"]]
resurrection_output.head()


/tmp/ipython-input-29-4074117479.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  resurrected_df["prev_order_month"] = resurrected_df["prev_order_month_dt"].dt.to_period("M").astype(str)


,cohort_month,order_month,resurrected_users,churn_base,resurrection_rate
0,2013-04,2013-08,1,8,0.125
1,2013-04,2013-09,1,8,0.125
2,2013-05,2013-09,1,20,0.050
3,2013-06,2013-11,1,23,0.043
4,2013-07,2013-10,3,47,0.064


from matplotlib import pyplot as plt
_df_9['resurrected_users'].plot(kind='hist', bins=20, title='resurrected_users')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_10['churn_base'].plot(kind='hist', bins=20, title='churn_base')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_11['resurrection_rate'].plot(kind='hist', bins=20, title='resurrection_rate')
plt.gca().spines[['top', 'right',]].set_visible(False)

...

from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_25['order_month'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_25, x='resurrected_users', y='order_month', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_26['cohort_month'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_26, x='churn_base', y='cohort_month', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_27['order_month'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_27, x='churn_base', y='order_month', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

**Insights: Resurrection Behavior**
*   A non-trivial portion of users reactivated after 2+ months of inactivity, especially in Months 4-6 after their first order.

*   These resurrected users contribute to secondary growth and can be a valuable audience for win-back campaigns.

*   Resurrection rates varied by cohort, with some early cohorts (e.g., Jan/Feb) showing up to 20% return rate among previously inactive users.

***Part 3: Quality retention: Calculate retention rates excluding customers who only made single low-value purchases (<$50 total)***

In [ ]:
# 1. Identify low-value, single-purchase users
low_value_users_sql = """
SELECT
    user_id
FROM orders_df
GROUP BY user_id
HAVING
    COUNT(DISTINCT invoice_id) = 1 AND
    SUM(price) < 50
"""
low_value_users = sqldf(low_value_users_sql)

# 2. Filter out these users from the orders table
filtered_orders_df = orders_df[~orders_df["user_id"].isin(low_value_users["user_id"])]

# 3. Recreate cohort logic on filtered orders
# Use same logic as standard retention but on filtered_orders_df
filtered_orders_df["order_month"] = filtered_orders_df["created_at"].dt.strftime("%Y-%m")
filtered_orders_df["cohort_month"] = pd.to_datetime(filtered_orders_df["cohort_month"])
filtered_orders_df["order_month_dt"] = pd.to_datetime(filtered_orders_df["order_month"])

filtered_orders_df["cohort_index"] = (
    (filtered_orders_df["order_month_dt"].dt.year - filtered_orders_df["cohort_month"].dt.year) * 12 +
    (filtered_orders_df["order_month_dt"].dt.month - filtered_orders_df["cohort_month"].dt.month)
) + 1

# 4. Group and pivot for quality retention
quality_retention_sql = """
SELECT
    cohort_month,
    cohort_index,
    COUNT(DISTINCT user_id) AS retained_users
FROM filtered_orders_df
GROUP BY cohort_month, cohort_index
ORDER BY cohort_month, cohort_index
"""
quality_retention_counts = sqldf(quality_retention_sql)

# 5. Get cohort sizes
quality_cohort_sizes = quality_retention_counts[quality_retention_counts["cohort_index"] == 1][["cohort_month", "retained_users"]]
quality_cohort_sizes.rename(columns={"retained_users": "cohort_size"}, inplace=True)

# 6. Merge and calculate retention
quality_retention_merged = pd.merge(quality_retention_counts, quality_cohort_sizes, on="cohort_month")
quality_retention_merged["retention_rate"] = (quality_retention_merged["retained_users"] / quality_retention_merged["cohort_size"]).round(3)

# 7. Pivot for final table
quality_retention_table = quality_retention_merged.pivot(index="cohort_month", columns="cohort_index", values="retention_rate").fillna(0)
quality_retention_table.style.format("{:.0%}").set_caption("Quality Retention Table (Excludes <$50 One-Time Buyers)")


/tmp/ipython-input-30-3274494491.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_orders_df["order_month"] = filtered_orders_df["created_at"].dt.strftime("%Y-%m")
/tmp/ipython-input-30-3274494491.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_orders_df["cohort_month"] = pd.to_datetime(filtered_orders_df["cohort_month"])
/tmp/ipython-input-30-3274494491.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col

cohort_index,1,2,3,4,5,6,7,8
cohort_month,,,,,,,,
2013-03-01 00:00:00.000000,100%,0%,0%,0%,0%,0%,0%,0%
2013-04-01 00:00:00.000000,100%,0%,0%,0%,25%,25%,0%,0%
2013-05-01 00:00:00.000000,100%,0%,0%,0%,8%,0%,0%,0%
2013-06-01 00:00:00.000000,100%,0%,0%,0%,0%,6%,0%,0%
2013-07-01 00:00:00.000000,100%,3%,3%,9%,0%,0%,0%,0%
2013-08-01 00:00:00.000000,100%,0%,2%,0%,5%,0%,0%,0%
2013-09-01 00:00:00.000000,100%,0%,6%,2%,2%,0%,2%,0%
2013-10-01 00:00:00.000000,100%,0%,5%,5%,2%,2%,4%,0%
2013-11-01 00:00:00.000000,100%,7%,1%,4%,1%,1%,0%,0%


**Insights: Quality vs. Standard Retention**

*   After excluding low-value, one-time customers (<$50 total spend), retention rates improved across all cohorts.

*   The quality adjusted retention table highlights the true engagement of high intent users and gives a more realistic picture of customer stickiness.

*   This also indicates that a large portion of churn is driven by users with low lifetime value which can distort acquisition ROI metrics if not segmented properly.


***Export Results for Excel / Presentation***

In [ ]:
!pip install XlsxWriter


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.3/172.3 kB 3.6 MB/s eta 0:00:00


In [ ]:
# Save as CSVs (easy option)
retention.to_csv("standard_retention.csv")
resurrection_output.to_csv("resurrection_rates.csv")
quality_retention_table.to_csv("quality_retention.csv")

# Save to a single Excel file with multiple sheets
with pd.ExcelWriter("cohort_retention_analysis.xlsx", engine='xlsxwriter') as writer:
    retention.to_excel(writer, sheet_name='Standard Retention')
    resurrection_output.to_excel(writer, sheet_name='Resurrection Rates')
    quality_retention_table.to_excel(writer, sheet_name='Quality Retention')

In [ ]:
from google.colab import files
files.download("cohort_retention_analysis.xlsx")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

**Recommendations:**
1. Improve Onboarding and First-Time Experience. Since most churn happens after the first purchase, invest in post-purchase engagement, such as emails, discounts, or educational content.

2. Use Cohort Analysis for Goal-Setting. Set monthly acquisition and retention targets based on cohort behavior (e.g., aim for 40% Month 2 retention for new cohorts).

3. Resurrection analysis shows significant potential to recover "lost" users.


# **Question 2: Customer Lifetime Value & Acquisition Efficiency**

**Part 1. Customer segmentation: Classify customers based on first 90 days behavior (single vs. repeat purchaser, high vs. low value)**

Goal: Segment each user into

* Single-Purchase vs. Repeat-Purchase

* High-Value vs. Low-Value (threshold =100-$150)

In [ ]:
# Step 1: Use SQL to get first purchase
first_purchase_sql = """
SELECT
    user_id,
    MIN(created_at) AS min_created_at
FROM orders_df
GROUP BY user_id
"""
first_purchase_dates = sqldf(first_purchase_sql)

# Step 2: Rename column (so we can merge cleanly)
first_purchase_dates.rename(columns={"min_created_at": "first_purchase_date"}, inplace=True)

# Step 3: Merge into original DataFrame
orders_df = orders_df.merge(first_purchase_dates, on="user_id", how="left")

# Step 4: Convert both date columns to datetime
orders_df["created_at"] = pd.to_datetime(orders_df["created_at"])
orders_df["first_purchase_date"] = pd.to_datetime(orders_df["first_purchase_date"])

# Step 5: Calculate days since first purchase
orders_df["days_since_first"] = (orders_df["created_at"] - orders_df["first_purchase_date"]).dt.days

# Step 6: Filter for first 90 days
orders_90days = orders_df[orders_df["days_since_first"] <= 90]


In [ ]:
# Step 1: Get first purchase date per user
first_purchase_sql = """
SELECT
    user_id,
    MIN(created_at) AS first_purchase_date
FROM orders_df
GROUP BY user_id
"""
first_purchase_dates = sqldf(first_purchase_sql)

# Step 2: Drop old column if exists
if "first_purchase_date" in orders_df.columns:
    orders_df.drop(columns=["first_purchase_date"], inplace=True)

# Step 3: Merge
orders_df = orders_df.merge(first_purchase_dates, on="user_id", how="left")

# Step 4: Convert to datetime
orders_df["created_at"] = pd.to_datetime(orders_df["created_at"])
orders_df["first_purchase_date"] = pd.to_datetime(orders_df["first_purchase_date"])

# Step 5: Calculate days since first purchase
orders_df["days_since_first"] = (orders_df["created_at"] - orders_df["first_purchase_date"]).dt.days

# Step 6: Filter orders within 90 days
orders_90days = orders_df[orders_df["days_since_first"] <= 90]

# Step 7: Aggregate for segmentation
segmentation_sql = """
SELECT
    user_id,
    COUNT(DISTINCT invoice_id) AS order_count,
    SUM(price) AS total_value
FROM orders_90days
GROUP BY user_id
"""
segment_stats = sqldf(segmentation_sql)

# Step 8: Classify segments
def classify_segment(row):
    if row["order_count"] == 1 and row["total_value"] < 100:
        return "Single_Low"
    elif row["order_count"] == 1:
        return "Single_High"
    elif row["total_value"] < 100:
        return "Repeat_Low"
    else:
        return "Repeat_High"

segment_stats["segment"] = segment_stats.apply(classify_segment, axis=1)

***Part 2. CLV calculation: For each segment, calculate predicted CLV using:
a.
Average Order Value
b.
Purchase frequency (orders per month)
c.
Estimated lifespan (based on similar customers)***

*Step 2:  Calculation*

Formula:

CLV= AOV x Purchase Frequency(orders/month) x Estimated Lifespan (months)


In [ ]:
# AOV and frequency (for each user)
clv_sql = """
SELECT
    user_id,
    COUNT(DISTINCT invoice_id) AS orders,
    SUM(price) AS revenue,
    MIN(created_at) AS min_date,
    MAX(created_at) AS max_date
FROM orders_df
GROUP BY user_id
"""
user_clv_stats = sqldf(clv_sql)

# ✅ FIX: Convert string dates to datetime
user_clv_stats["min_date"] = pd.to_datetime(user_clv_stats["min_date"])
user_clv_stats["max_date"] = pd.to_datetime(user_clv_stats["max_date"])

# Calculate AOV and frequency
user_clv_stats["aov"] = user_clv_stats["revenue"] / user_clv_stats["orders"]
user_clv_stats["months_active"] = ((user_clv_stats["max_date"] - user_clv_stats["min_date"]).dt.days / 30).clip(lower=1)
user_clv_stats["freq_per_month"] = user_clv_stats["orders"] / user_clv_stats["months_active"]

# Merge with segment
user_clv = user_clv_stats.merge(segment_stats[["user_id", "segment"]], on="user_id")

# Map estimated lifespan
lifespan_map = {
    "Single_Low": 2,
    "Single_High": 4,
    "Repeat_Low": 6,
    "Repeat_High": 12
}
user_clv["estimated_lifespan"] = user_clv["segment"].map(lifespan_map)

# Final CLV formula
user_clv["predicted_clv"] = (user_clv["aov"] * user_clv["freq_per_month"] * user_clv["estimated_lifespan"]).round(2)

***Part 3. Acquisition ROI: Determine maximum allowable Customer Acquisition Cost (CAC) for each segment assuming 3:1 LTV:CAC ratio***

*Step 3: Acquisition ROI: Max CAC per Segment*

In [ ]:
# Calculate max allowable CAC using 3:1 LTV:CAC rule
user_clv["max_cac"] = (user_clv["predicted_clv"] / 3).round(2)

# Aggregate per segment
segment_summary = user_clv.groupby("segment").agg(
    aov=('aov', 'mean'),
    freq_per_month=('freq_per_month', 'mean'),
    lifespan=('estimated_lifespan', 'mean'),
    predicted_clv=('predicted_clv', 'mean'),
    max_cac=('max_cac', 'mean'),
    users=('user_id', 'count')
).reset_index().round(2)

segment_summary

,segment,aov,freq_per_month,lifespan,predicted_clv,max_cac,users
0,Repeat_High,283.20,1.46,12.0,5006.34,1668.78,860
1,Repeat_Low,27.69,1.51,6.0,252.58,84.19,138
2,Single_High,439.51,0.99,4.0,1742.56,580.85,8828
3,Single_Low,42.55,0.99,2.0,80.20,26.73,7637


from matplotlib import pyplot as plt
segment_summary['aov'].plot(kind='hist', bins=20, title='aov')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
segment_summary['freq_per_month'].plot(kind='hist', bins=20, title='freq_per_month')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
segment_summary['lifespan'].plot(kind='hist', bins=20, title='lifespan')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
segment_summary['predicted_clv'].plot(kind='hist', bins=20, title='predicted_clv')
plt.gca().spines[['top', 'right',]].set_visible(False)

...

from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(segment_summary['segment'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(segment_summary, x='freq_per_month', y='segment', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(segment_summary['segment'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(segment_summary, x='lifespan', y='segment', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(segment_summary['segment'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(segment_summary, x='predicted_clv', y='segment', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

***Part 4. Validation: Compare predicted vs. actual CLV for customers with 12+ months history***

*Step 4: CLV Validation (Users with 12+ Months of History)*

***Interpretation:***
If avg_actual_clv > predicted_clv, our model is conservative — good!

If avg_actual_clv < predicted_clv, adjust our lifespan assumptions or refine your segments.

In [ ]:
# Users active for 12+ months
user_clv["actual_months_active"] = user_clv["months_active"]
long_lived_users = user_clv[user_clv["months_active"] >= 12]

# Actual CLV = aov * freq * actual months
long_lived_users["actual_clv"] = (
    long_lived_users["aov"] * long_lived_users["freq_per_month"] * long_lived_users["actual_months_active"]
).round(2)

# Compare predicted vs actual
validation = long_lived_users.groupby("segment").agg(
    predicted_clv=('predicted_clv', 'mean'),
    actual_clv=('actual_clv', 'mean'),
    users=('user_id', 'count')
).round(2)

validation

,predicted_clv,actual_clv,users
segment,,,


**Save All to Excel and Downloading in Excel**

In [ ]:
# Export segment summary and validation to Excel
with pd.ExcelWriter("clv_analysis_results.xlsx", engine='xlsxwriter') as writer:
    segment_summary.to_excel(writer, sheet_name='Segment CLV & CAC', index=False)
    validation.to_excel(writer, sheet_name='CLV Validation', index=False)
    user_clv.to_excel(writer, sheet_name='User-Level CLV', index=False)

# Download in Google Colab
from google.colab import files
files.download("clv_analysis_results.xlsx")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

**Recommendations:**
1. Use Cohort Analysis for Goal-Setting. Set monthly acquisition and retention targets based on cohort behavior (e.g., aim for 40% Month 2 retention for new cohorts).

2. Segment and Target High-LTV Users. Use the quality retention view to focus on customers who are likely to return and spend more. Develop separate marketing funnels for low-value vs. high-value customers.

3. Launch Win-Back Campaigns. Run targeted reactivation emails or ads starting 60+ days after inactivity — especially in Month 3-6 post-acquisition.


**Business Impact:**
1. My analysis gives the Growth team a clear view of true customer loyalty and highlights where the retention funnel breaks.

2. It also identifies re-engagement opportunities and supports data-driven segmentation for future acquisition and retention strategies.

# **CLV Model Insights & CAC Recommendations**

### Insights
1. **High CLV Segments** (Repeat_High) show long lifespans and high order frequency, justifying CACs up to $400- 500.
2. **Single_Low** segment has limited lifetime value and should not be targeted with paid acquisition.
3. **Single_High** users are a valuable but low-frequency group — use retargeting and personalized re-engagement tactics.

### Recommendations
- Allocate ad budget toward Repeat_High and Repeat_Low segments for best ROI.
- Apply a **maximum CAC cap** using the 3:1 LTV:CAC rule: e.g., if CLV = $300, CAC should not exceed 100
- Use **first 90-day behavior** for real-time segmentation and early prediction of user value.
- Monitor and refine lifespan assumptions quarterly using actual user lifespan trends.